In [1]:
import calendar
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
    'qty':'{:,}','price':'{:.2f}','amount':'{:,.2f}'
}
cols = 'trade name qty price active'.split()

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2022, 8, 20), datetime.date(2022, 8, 19))

### Yesterday must be last business day

In [2]:
#yesterday = date(2021, 8, 11)
yesterday

datetime.date(2022, 8, 19)

In [3]:
format_dict = {
    'qty':'{:,}',
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%',  
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',    
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
}

### Matching check with profits in portmy database with 2022-q2 data 

In [4]:
sql = '''
SELECT *
FROM stocks
ORDER BY market, name'''
stocks = pd.read_sql(sql, conlite)
stocks.shape

(56, 18)

In [5]:
sql = '''
SELECT *
FROM profits
ORDER BY name'''
profits = pd.read_sql(sql, conmy)
profits.shape

(40, 23)

In [6]:
df_merge = pd.merge(stocks, profits, on='name', how='outer', indicator=True)
df_merge.shape

(60, 41)

In [7]:
names = df_merge[df_merge._merge == 'right_only']['name']
names

56      BCT
57      CRC
58      KSL
59    OISHI
Name: name, dtype: object

In [8]:
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'BCT', 'CRC', 'KSL', 'OISHI'"

In [9]:
sql = """
SELECT Null AS id, name, market, max_price, min_price, daily_volume AS volume, beta 
FROM stocks
WHERE name IN (%s)"""
sql = sql % in_p
my_stocks = pd.read_sql(sql, conmy)
my_stocks

,id,name,market,max_price,min_price,volume,beta
0,None,BCT,SET,68.50,44.75,1.40,0.50
1,None,CRC,SET50 / SETCLMV / SETTHSI / SETWB,41.75,31.50,483.40,1.43
2,None,KSL,sSET / SETCLMV,4.40,3.22,37.54,1.02
3,None,OISHI,SET,50.50,41.25,0.94,0.60


In [10]:
my_stocks.dtypes

id            object
name          object
market        object
max_price    float64
min_price    float64
volume       float64
beta         float64
dtype: object

In [12]:
my_stocks['status'] = 'O'
my_stocks['buy_target'] = 0.00
my_stocks['sell_target'] = 0.00
my_stocks['cost'] = 0.00
my_stocks['qty'] = 0
my_stocks['buy_spread'] = -4
my_stocks['sell_spread'] = 4
my_stocks['available_qty'] = 0
my_stocks['bl'] = 0
my_stocks['sh'] = 0
my_stocks['reason'] = 'XXX'
my_stocks

,id,name,market,max_price,min_price,volume,beta,status,buy_target,sell_target,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason
0,None,BCT,SET,68.50,44.75,1.40,0.50,O,0.0,0.0,0.0,0,-4,4,0,0,0,XXX
1,None,CRC,SET50 / SETCLMV / SETTHSI / SETWB,41.75,31.50,483.40,1.43,O,0.0,0.0,0.0,0,-4,4,0,0,0,XXX
2,None,KSL,sSET / SETCLMV,4.40,3.22,37.54,1.02,O,0.0,0.0,0.0,0,-4,4,0,0,0,XXX
3,None,OISHI,SET,50.50,41.25,0.94,0.60,O,0.0,0.0,0.0,0,-4,4,0,0,0,XXX


In [13]:
rearrange_col = 'id name max_price min_price status buy_target sell_target volume beta cost qty buy_spread sell_spread available_qty bl sh reason market'.split()
rearrange_col

['id',
 'name',
 'max_price',
 'min_price',
 'status',
 'buy_target',
 'sell_target',
 'volume',
 'beta',
 'cost',
 'qty',
 'buy_spread',
 'sell_spread',
 'available_qty',
 'bl',
 'sh',
 'reason',
 'market']

In [14]:
my_stocks = my_stocks[rearrange_col]
my_stocks

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,None,BCT,68.50,44.75,O,0.0,0.0,1.40,0.50,0.0,0,-4,4,0,0,0,XXX,SET
1,None,CRC,41.75,31.50,O,0.0,0.0,483.40,1.43,0.0,0,-4,4,0,0,0,XXX,SET50 / SETCLMV / SETTHSI / SETWB
2,None,KSL,4.40,3.22,O,0.0,0.0,37.54,1.02,0.0,0,-4,4,0,0,0,XXX,sSET / SETCLMV
3,None,OISHI,50.50,41.25,O,0.0,0.0,0.94,0.60,0.0,0,-4,4,0,0,0,XXX,SET


In [15]:
rcds = my_stocks.values.tolist()
len(rcds)

4

In [16]:
for rcd in rcds:
    print(rcd)

[None, 'BCT', 68.5, 44.75, 'O', 0.0, 0.0, 1.4, 0.5, 0.0, 0, -4, 4, 0, 0, 0, 'XXX', 'SET    ']
[None, 'CRC', 41.75, 31.5, 'O', 0.0, 0.0, 483.4, 1.43, 0.0, 0, -4, 4, 0, 0, 0, 'XXX', 'SET50 / SETCLMV / SETTHSI / SETWB    ']
[None, 'KSL', 4.4, 3.22, 'O', 0.0, 0.0, 37.54, 1.02, 0.0, 0, -4, 4, 0, 0, 0, 'XXX', 'sSET / SETCLMV    ']
[None, 'OISHI', 50.5, 41.25, 'O', 0.0, 0.0, 0.94, 0.6, 0.0, 0, -4, 4, 0, 0, 0, 'XXX', 'SET    ']


In [17]:
for rcd in rcds:
    conlite.execute("""INSERT INTO stocks (id, name, max_price, min_price, status, buy_target, sell_target, volume, beta, cost, qty, buy_spread, sell_spread, available_qty, bl, sh, \
reason, market) \
VALUES( ?, ?, ?, ?, ?, ?, ?, ?, ?,?, ?, ?, ?, ?, ?, ?, ?, ?)""", rcd)

### End of Process insert profit records to stocks

### Process for stock records that are not in profits table

In [18]:
sql = '''
SELECT *
FROM stocks
ORDER BY market, name'''
stocks = pd.read_sql(sql, conlite)
stocks.shape

(60, 18)

In [19]:
sql = '''
SELECT *
FROM profits
ORDER BY name'''
profits = pd.read_sql(sql, conmy)
profits.shape

(40, 23)

In [20]:
df_merge2 = pd.merge(stocks, profits, on='name', how='outer', indicator=True)
df_merge2.shape

(60, 41)

In [21]:
names = df_merge2[df_merge2._merge == 'left_only']['name']
names

1        ASP
4        DCC
6        DIF
8      JASIF
11     MAKRO
12       MCS
13       NER
16      SCCC
17      SENA
21       TMT
24     WHAIR
25     WHART
30       BCH
33    DOHOME
37       RCL
41       STA
43     SYNEX
51       KCE
54     PTTGC
55       SCC
Name: name, dtype: object

In [22]:
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'ASP', 'DCC', 'DIF', 'JASIF', 'MAKRO', 'MCS', 'NER', 'SCCC', 'SENA', 'TMT', 'WHAIR', 'WHART', 'BCH', 'DOHOME', 'RCL', 'STA', 'SYNEX', 'KCE', 'PTTGC', 'SCC'"

In [23]:
sql = '''
SELECT *
FROM stocks
WHERE name IN (%s)
ORDER BY market, name'''
sql = sql % in_p
lite_stocks = pd.read_sql(sql, conlite)
lite_stocks[['market','name']]

,market,name
0,SET,ASP
1,SET,DCC
2,SET,DIF
3,SET,JASIF
4,SET,MAKRO
5,SET,MCS
6,SET,NER
7,SET,SCCC
8,SET,SENA
9,SET,TMT


In [24]:
sql = '''
SELECT name
FROM buy
ORDER BY name'''
stock_buy = pd.read_sql(sql, const)
stock_buy

,name
0,ASP
1,BCH
2,CPNREIT
3,DCC
4,DIF
5,DOHOME
6,JASIF
7,KCE
8,MAKRO
9,MCS


In [25]:
df_merge3 = pd.merge(lite_stocks, stock_buy, on='name', how='outer', indicator=True)
df_merge3

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market,_merge
0,699.0,ASP,4.24,2.62,T,3.00,3.82,70.06,0.63,3.80,10000.0,-9.0,12.0,30000.0,0.0,0.0,52WL,SET,both
1,728.0,DCC,0.00,0.00,U,0.00,2.98,0.00,0.00,2.96,60000.0,-4.0,4.0,60000.0,0.0,0.0,CP1S,SET,both
2,325.0,DIF,14.50,11.70,I,13.20,14.60,147.37,0.39,14.70,10000.0,-5.0,5.0,40000.0,0.0,0.0,CUT,SET,both
3,300.0,JASIF,11.30,9.45,T,8.60,10.10,173.61,0.32,10.00,10000.0,-5.0,4.0,130000.0,0.0,0.0,RD10pct,SET,both
4,704.0,MAKRO,57.25,35.00,U,0.00,39.00,618.00,1.00,40.40,1500.0,-8.0,8.0,7500.0,0.0,0.0,C40.40,SET,both
5,85.0,MCS,16.20,11.70,B,10.60,15.50,37.02,0.50,15.40,5000.0,-5.0,5.0,75000.0,6.0,3.0,52WL,SET,both
6,602.0,NER,8.65,5.50,U,0.00,7.10,116.83,0.50,7.45,9000.0,-13.0,14.0,27000.0,0.0,0.0,P50%,SET,both
7,701.0,SCCC,188.50,150.00,T,148.50,0.00,57.87,0.61,171.00,300.0,-10.0,9.0,1200.0,0.0,0.0,RD10pct,SET,both
8,711.0,SENA,5.50,3.44,T,3.68,4.50,31.56,1.05,4.48,15000.0,-8.0,10.0,105000.0,0.0,0.0,CP1S,SET,both
9,645.0,TMT,15.00,9.70,B,7.90,9.00,13.30,1.01,9.90,12000.0,-5.0,11.0,48000.0,0.0,0.0,RD20pct,SET,both


In [26]:
names = df_merge3[df_merge3._merge == 'left_only']['name']
names

Series([], Name: name, dtype: object)

In [27]:
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

''

In [28]:
sqlDel = """
DELETE FROM stocks
WHERE name IN (%s)"""
sqlDel = sqlDel % in_p
print(sqlDel)


DELETE FROM stocks
WHERE name IN ()


In [29]:
rp = conlite.execute(sqlDel)
rp.rowcount

0

### End of Process to delete stock record that are not in profits and avalable_qty = 0

In [24]:
sql = '''
SELECT *
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.dtypes

name                object
q4                 float64
q3                 float64
q2                 float64
q1                 float64
dividend           float64
shares               int64
xdate       datetime64[ns]
paiddate    datetime64[ns]
kind                object
actual               int64
dtype: object

In [25]:
dividend.sort_values(['xdate'],ascending=[True]).style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
5,HTC,1.120000,0.000000,0.830000,0.000000,1.950000,0,2021-09-23 00:00:00,2021-09-23 00:00:00,,0
67,LPF,0.171900,0.189400,0.179300,0.187800,0.728400,0,2022-01-28 00:00:00,2022-02-15 00:00:00,,0
32,GLOBAL,0.000000,0.000000,0.000000,0.000000,0.000000,0,2022-02-02 00:00:00,2022-02-02 00:00:00,,0
70,IP,0.000000,0.000000,0.000000,0.000000,0.000000,0,2022-02-02 00:00:00,2022-02-02 00:00:00,,0
76,DOHOME,0.000000,0.000000,0.000000,0.000000,0.000000,7200,2022-02-02 00:00:00,2022-02-02 00:00:00,,0
79,STARK,0.000000,0.000000,0.000000,0.000000,0.000000,0,2022-02-02 00:00:00,2022-02-02 00:00:00,,0
80,PTTEP,3.000000,0.000000,2.000000,0.000000,5.000000,0,2022-02-10 00:00:00,2022-04-18 00:00:00,,0
77,SCB,2.630000,0.000000,1.430000,0.000000,4.060000,0,2022-02-14 00:00:00,2022-04-22 00:00:00,,0
41,ADVANC,4.240000,0.000000,3.450000,0.000000,7.690000,0,2022-02-18 00:00:00,2022-04-19 00:00:00,,0
44,GVREIT,0.195000,0.176500,0.176500,0.191500,0.739500,0,2022-02-23 00:00:00,2022-09-03 00:00:00,,0


In [26]:
df_merge = pd.merge(stocks, dividend, on='name', how='outer', indicator=True)
left_mask= df_merge._merge == 'left_only'
df_merge[left_mask]
#no_dividend.sort_values(['xdate'],ascending=[True])

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,...,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual,_merge
0,9011.0,AIT,9.80,4.12,O,0.0,0.00,140.03,1.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
5,9709.0,COTTO,3.18,1.55,O,0.0,0.00,10.15,0.87,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
8,9713.0,FORTH,53.00,7.70,O,0.0,0.00,188882.96,0.25,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
9,9188.0,GGC,14.40,9.60,O,0.0,0.00,9.31,0.96,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
19,9449.0,SIS,48.75,25.75,O,30.5,0.00,33.27,1.75,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
20,9455.0,SMPC,14.20,10.00,O,14.0,0.00,14.45,0.66,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
22,9714.0,SSP,14.27,9.10,O,0.0,0.00,35776.45,0.24,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
28,9053.0,BCPG,15.40,11.10,O,0.0,0.00,157.15,0.95,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
29,9074.0,BPP,20.30,15.30,O,0.0,0.00,28.02,0.83,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
31,9165.0,ESSO,8.90,7.00,O,0.0,0.00,173.92,1.03,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only


### End of Process

### Consensus table

In [ ]:
sql = '''
SELECT * FROM consensus ORDER BY name'''
css = pd.read_sql(sql, conlite)
css

In [ ]:
sql = '''
SELECT *
FROM profits
ORDER BY name'''
df_prf = pd.read_sql(sql, conmy)
df_prf.shape

In [ ]:
sql = '''
SELECT *
FROM stocks
ORDER BY market, name'''
df_stk = pd.read_sql(sql, conlite)
df_stk.shape

In [ ]:
df_merge = pd.merge(df_prf,df_stk,on='name',how='outer',indicator=True)
df_merge.shape

In [ ]:
left_mask= df_merge._merge == 'left_only'
df_merge[left_mask]

In [ ]:
right_mask= df_merge._merge == 'right_only'
df_merge[right_mask]

In [ ]:
both_mask= df_merge._merge == 'both'
df_merge[both_mask]

In [ ]:
sql = '''
SELECT * FROM SALES ORDER BY name'''
sales = pd.read_sql(sql, conlite)
sales.shape[0]

In [ ]:
sql = '''
DELETE FROM sales
WHERE name = "TCAP"'''
rp = conlite.execute(sql)
rp.rowcount

### Temp process

In [37]:
sql = """
SELECT market,name,buy_target,qty
FROM stocks
WHERE buy_target = 0 AND status IN ('B','O')
ORDER BY market, name"""
df_stocks = pd.read_sql(sql, conlite)
df_stocks

,market,name,buy_target,qty
0,SET,CPNCG,0,0
1,SET,CPNREIT,0,0
2,SET,JWD,0,0
3,SET,LANNA,0,0
4,SET,PTL,0,0
5,SET,TFFIF,0,0
6,SET,VNG,0,0
7,SET100,AMATA,0,0
8,SET100,BAM,0,0
9,SET100,BCP,0,0


### Process to set target price

#### 1) Set50 records

In [44]:
pd.read_sql_query('SELECT * FROM stocks WHERE market = "SET50" AND status = "O" ORDER BY name', conlite)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9712,BANPU,14.90,9.65,O,11.50,0,1316230.44,1.52,0.0,12000,-4,4,0,0,0,1L,SET50
1,9722,BLA,48.00,26.00,O,37.00,0,285.38,0.88,0.0,4000,-4,4,0,0,0,XXX,SET50
2,9231,IVL,52.00,36.00,O,42.00,0,1112.71,1.41,48.5,3000,-11,11,0,0,0,13WL,SET50
3,9727,JMART,64.00,30.50,O,43.00,0,521.57,1.65,0.0,3000,-4,4,0,0,0,XXX,SET50
4,9733,PTT,41.25,32.75,O,34.00,0,2188.39,1.00,0.0,4500,-4,4,0,0,0,XXX,SET50
5,9717,TTB,0.00,0.00,O,0.95,0,0.00,0.00,0.0,90000,-4,4,0,0,0,52WL,SET50


In [43]:
name = 'PTT'
limit = 20 # 1 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,20.00,20.00,20.00,20.00,20.00
mean,34.95,35.14,34.42,47847446.95,34.71
std,1.02,0.98,0.93,19277106.16,0.99
min,33.50,34.00,33.00,19667139.00,33.25
25%,34.25,34.50,33.94,31321828.00,34.19
50%,34.62,34.75,34.25,46988327.50,34.38
75%,35.31,35.50,34.81,59450079.50,35.25
max,37.00,37.00,36.25,88922626.00,37.00


#### 2) Set100 records

In [50]:
pd.read_sql_query('SELECT * FROM stocks WHERE market = "SET100" AND status = "O" ORDER BY name', conlite)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9719,AMATA,23.00,17.1,O,17,0,169.77,1.56,0,6000,-4,4,0,0,0,XXX,SET100
1,9720,BAM,22.30,15.9,O,16,0,405.08,1.30,0,6000,-4,4,0,0,0,XXX,SET100
2,9721,BCP,35.25,23.3,O,29,0,264.08,0.94,0,3000,-4,4,0,0,0,XXX,SET100
3,9736,STARK,5.45,3.9,O,4,0,134.56,0.90,0,25000,-4,4,0,0,0,XXX,SET100


In [49]:
name = 'STARK'
limit = 60 # 3 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,60.00,60.00,60.00,6.000000e+01,60.00
mean,4.42,4.50,4.37,2.817365e+07,4.44
std,0.27,0.28,0.27,3.506276e+07,0.28
min,4.00,4.14,3.90,3.313251e+06,4.02
25%,4.22,4.30,4.18,9.217222e+06,4.22
50%,4.34,4.41,4.31,1.462364e+07,4.35
75%,4.57,4.67,4.51,2.773165e+07,4.58
max,5.05,5.10,4.92,1.688560e+08,5.05


#### 3) Set records

In [51]:
pd.read_sql_query('SELECT * FROM stocks WHERE market = "SET" AND status = "O" ORDER BY name', conlite)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9724,CPNCG,12.80,10.40,O,0.00,0,3.15,0.49,0,0,-4,4,0,0,0,XXX,SET
1,9725,CPNREIT,22.40,17.90,O,0.00,0,21.29,0.47,0,0,-4,4,0,0,0,XXX,SET
2,9729,JWD,24.40,12.50,O,0.00,0,96.10,1.45,0,0,-4,4,0,0,0,XXX,SET
3,9730,LANNA,29.00,15.00,O,0.00,0,111.34,0.86,0,0,-4,4,0,0,0,XXX,SET
4,9732,PTL,30.50,20.60,O,0.00,0,53.78,0.81,0,0,-4,4,0,0,0,XXX,SET
5,9714,SSP,14.27,9.10,O,8.75,0,35776.45,0.24,0,10000,-9,6,0,0,0,52WL,SET
6,9739,TFFIF,8.50,7.00,O,0.00,0,24.39,0.41,0,0,-4,4,0,0,0,XXX,SET
7,9740,VNG,10.10,7.05,O,0.00,0,17.16,0.73,0,0,-4,4,0,0,0,XXX,SET


In [59]:
name = 'VNG'
limit = 120 # 6 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,120.00,120.00,120.00,120.00,120.00
mean,7.66,7.79,7.58,2248783.37,7.69
std,0.37,0.39,0.35,2150419.52,0.37
min,7.10,7.20,7.10,125400.00,7.20
25%,7.35,7.45,7.30,707225.75,7.40
50%,7.55,7.70,7.45,1490555.00,7.57
75%,7.95,8.15,7.85,2832852.25,8.00
max,8.55,8.75,8.45,10934861.00,8.55
